In [1]:
# Import standard data and visualization libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# It seems there was an issue recently relating to tensorflow and Keras that broke the Top2Vec import: https://github.com/tensorflow/probability/issues/1795
# We attempt the workaround detailed here: https://github.com/tensorflow/tensorflow/releases/tag/v2.16.1
!pip install tf-keras~=2.16
import os
os.environ["TF_USE_LEGACY_KERAS"]="1"
import tf_keras as keras

# Import the Top2Vec
# This also requires installation on Colaboratory, as it's not pre-installed (😢)
!pip install top2vec[sentence_encoders]
from top2vec import Top2Vec

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 12.5 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successfully uninstalled h5py-3.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
      Su

In [ ]:
from wordcloud import STOPWORDS

In [7]:
df = pd.read_csv('https://raw.githubusercontent.com/MiriamSchirmer/genocide-transcript-corpus/main/Dataset_GTC-V2.csv', sep =';')
df = df.loc[df['tribunal']=='ECCC']
df = df.loc[df['role'].str.contains('Accused')]
df

,tribunal,id_transcript,case,accused,date,text,trauma,role,witnesses,n_witnesses,start,id_annotation,id_document,url
17053,ECCC,E1/12.1,Case001,Kaing Guev Eav,2009-04-20 00:00:00 UTC,"THE ACCUSED: Your Honours, first of all, let m...",0,Accused,"[Chan Voeun (KW-31), Chan Khan (KW-32)]",2,24534,63b1f85497ad59b4cfc57a2d,63a60e2397ad59b4cfc571ec,http://www.eccc.gov.kh/sites/default/files/doc...
17055,ECCC,E1/12.1,Case001,Kaing Guev Eav,2009-04-20 00:00:00 UTC,THE ACCUSED: I can state that Chan Voeun was n...,0,Accused,"[Chan Voeun (KW-31), Chan Khan (KW-32)]",2,25704,63b1f85c97ad59b4cfc57a2e,63a60e2397ad59b4cfc571ec,http://www.eccc.gov.kh/sites/default/files/doc...
17056,ECCC,E1/12.1,Case001,Kaing Guev Eav,2009-04-20 00:00:00 UTC,"When Mr. Pon, Comrade Pon ask Comrade Meas how...",0,Accused,"[Chan Voeun (KW-31), Chan Khan (KW-32)]",2,26782,63b1f86097ad59b4cfc57a2f,63a60e2397ad59b4cfc571ec,http://www.eccc.gov.kh/sites/default/files/doc...
17057,ECCC,E1/12.1,Case001,Kaing Guev Eav,2009-04-20 00:00:00 UTC,In 1973 they did not yet assign the cooperativ...,0,Accused,"[Chan Voeun (KW-31), Chan Khan (KW-32)]",2,27834,63b1f86697ad59b4cfc57a30,63a60e2397ad59b4cfc571ec,http://www.eccc.gov.kh/sites/default/files/doc...
17059,ECCC,E1/12.1,Case001,Kaing Guev Eav,2009-04-20 00:00:00 UTC,"THE ACCUSED: Your Honours, this person is not ...",0,Accused,"[Chan Voeun (KW-31), Chan Khan (KW-32)]",2,29297,63b1f87197ad59b4cfc57a32,63a60e2397ad59b4cfc571ec,http://www.eccc.gov.kh/sites/default/files/doc...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32681,ECCC,E1/42.1,Case001,Kaing Guev Eav,2009-07-02 00:00:00 UTC,So these are my observations on the two separa...,0,Accused,[Norng Chanphal],1,103429,63bddea697ad59b4cfc5ba65,63a60e2797ad59b4cfc5721a,http://www.eccc.gov.kh/sites/default/files/doc...
32691,ECCC,E1/42.1,Case001,Kaing Guev Eav,2009-07-02 00:00:00 UTC,"THE ACCUSED: Your Honour, I do not object to t...",0,Accused,[Norng Chanphal],1,107288,63bddf0997ad59b4cfc5ba70,63a60e2797ad59b4cfc5721a,http://www.eccc.gov.kh/sites/default/files/doc...
32694,ECCC,E1/42.1,Case001,Kaing Guev Eav,2009-07-02 00:00:00 UTC,THE ACCUSED: That's correct. Probably his mot...,0,Accused,[Norng Chanphal],1,108574,63bddf1797ad59b4cfc5ba73,63a60e2797ad59b4cfc5721a,http://www.eccc.gov.kh/sites/default/files/doc...
32700,ECCC,E1/42.1,Case001,Kaing Guev Eav,2009-07-02 00:00:00 UTC,"THE ACCUSED: Mr. President, all evidence that ...",0,Accused,[Norng Chanphal],1,110682,63bddf3f97ad59b4cfc5ba79,63a60e2797ad59b4cfc5721a,http://www.eccc.gov.kh/sites/default/files/doc...


In [8]:
def drop(text): #Getting rid of names at beginning
    if ": " in text:
        return text[text.index(":")+2:]
    else:
        return text

def q(text): #Getting rid of the text being prefaced with Q.
  if "Q." in text:
    return text[text.index("Q.")+3:]
  else:
    return text

def a(text): #Getting rid of the text being prefaced with A.
  if "A." in text:
    return text[text.index("A.")+3:]
  else:
    return text

def interpretation(text): #Getting rid of the bracketed interpretation section
  if "[Interpretation]" in text:
    return text[text.index("[Interpretation]")+16:]
  else:
    return text

def strip(text): #Getting rid of white space
  return text.strip()


df["text"] = df["text"].apply(interpretation)
df["text"] = df["text"].apply(a)
df["text"] = df["text"].apply(q)

df["text"] = df["text"].apply(drop)
df["text"] = df["text"].apply(strip)

In [ ]:
my_stop = list(STOPWORDS) #Using our own custom stopwords, maybe it will help?
my_stop.append('„')
my_stop.append('“')
my_stop.append('’')
my_stop.append('ðž')
my_stop.append('ð¹å¬')
my_stop.append('ðµä•ð')
my_stop.append('ðµä')
my_stop.append('ð')
my_stop.append('î')
my_stop.append('ëå†ä¯')
my_stop.append('ë')
my_stop.append('å†ä¯')
my_stop.append('åÿ')
my_stop.append('å¬')
my_stop.append('åªë')
my_stop.append('åªå†ð¹â…œåµå™ðµä')
my_stop.append('åªä®ð¹å†î‰ëâ')
my_stop.append('åªä®ð¹')
my_stop.append('åªä®')
my_stop.append('ää³ðš')
my_stop.append('äâ‚§åÿ	')
my_stop.append('äâ‚§')
my_stop.append('ä')
my_stop.append('ã‰lisabeth	')
my_stop.append('â€')
my_stop.append('äâ‚§åÿ')
my_stop.append('ã‰lisabeth')
my_stop.append('â…œâ‚¤äœ')
my_stop.append('â‚£ã°ä')
my_stop.append('â‚£')
my_stop.append('®åªä³')
my_stop.append('c')
my_stop.append('kw')

In [9]:
documents = list(df["text"]) #Convert the text column to a list
model = Top2Vec(documents, embedding_model='universal-sentence-encoder')

2024-04-12 18:42:10,385 - top2vec - INFO - Pre-processing documents for training
INFO:top2vec:Pre-processing documents for training
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
2024-04-12 18:42:10,463 - top2vec - INFO - Downloading universal-sentence-encoder model
INFO:top2vec:Downloading universal-sentence-encoder model
2024-04-12 18:42:17,689 - top2vec - INFO - Creating joint document/word embedding
INFO:top2vec:Creating joint document/word embedding
2024-04-12 18:42:18,700 - top2vec - INFO - Creating lower dimension embedding of documents
INFO:top2vec:Creating lower dimension embedding of documents
2024-04-12 18:42:25,981 - top2vec - INFO - Finding dense areas of documents
INFO:top2vec:Finding dense areas of documents
2024-04-12 18:42:25,994 - top2vec - INFO - Finding topics
INFO:top2vec:Finding topics


In [10]:
model.get_num_topics()

2

In [11]:
topic_sizes, topic_nums = model.get_topic_sizes()
topic_sizes

array([147,  55])

In [12]:
topic_words, word_scores, topic_nums = model.get_topics(2)
topic_words

array([['testimony', 'had', 'who', 'in', 'was', 'about', 'mr', 'on',
        'person', 'were', 'president', 'said', 'so', 'also', 'did', 'of',
        'as', 'there', 'not', 'have', 'from', 'this', 'at', 'with', 'to',
        'when', 'by', 'is', 'the', 'that', 'me', 'for', 'he', 'all',
        'him', 'and', 'people', 'but', 'it', 'his', 'be', 'or', 'you',
        'my', 'would', 'they', 'time', 'like', 'because'],
       ['testimony', 'had', 'in', 'was', 'about', 'who', 'on', 'from',
        'person', 'as', 'mr', 'so', 'at', 'he', 'president', 'by', 'of',
        'were', 'not', 'did', 'him', 'with', 'to', 'the', 'there',
        'have', 'when', 'is', 'for', 'that', 'all', 'it', 'his', 'be',
        'this', 'my', 'me', 'also', 'or', 'you', 'said', 'but', 'people',
        'they', 'and', 'because', 'like', 'time', 'would']], dtype='<U9')

In [17]:
topic_words, word_scores, topic_scores, topic_nums = model.search_topics(keywords=["prisoner"], num_topics=3)
topic_nums

ValueError: 'prisoner' has not been learned by the model so it cannot be searched.

In [18]:
documents, document_scores, document_ids = model.search_documents_by_topic(topic_num=1, num_docs=10)
documents

array(["I very well know the situation of the pedagological institutions, and I knew that Pol Pot realized that the professor was working in that institute and Chao Seng (phonetic) was the chairman of the association and Mai Man (phonetic) was the professor at that school; actually, he was director of that university.  And Sous Nou alias Chou (phonetic), was also a senior professor at that school and later on he was the Secretary of Sector 24. So Son Sen clearly ordered me to monitor the activity of Sous Nou alias Chou (phonetic), and through my monitoring activities I learned that he was not implicated in that matter. So the fact that if I knew the professor was detained there -- if I knew that and the interrogation would then focus on all these links but, as I said, I did not know he was detained.  That's why there was no annotation that I made on the confession. And this is my response.",
       "And the third point or quality of Chao Seng was that he established the pedagogical ins